In [1]:
import yaml
from models import *
import os

## Configuration



In [2]:
storage_class_rwo = "managed-nfs-storage"
storage_class_rwx = "managed-nfs-storage"

workspace_volume_size = "50Gi"
calrissian_volume_size = "50Gi"

## Volumes

Create the Volumes

### Workspace Volume

The workspace volume is persisted.

In [3]:
workspace_volume = Volume(
    name="workspace-volume",
    size=workspace_volume_size,
    claim_name="workspace-claim",
    mount_path="/workspace",
    storage_class=storage_class_rwo,
    access_modes=["ReadWriteOnce"],
    volume_mount=VolumeMount(name="workspace-volume", mount_path="/workspace"),
    persist=False,
)

### Calrissian Volume

This is a RWX volume, not persisted

In [4]:
calrissian_volume = Volume(
    name="calrissian-volume",
    claim_name="calrissian-claim",
    size=calrissian_volume_size,
    storage_class=storage_class_rwx,
    access_modes=["ReadWriteMany"],
    volume_mount=VolumeMount(name="calrissian-volume", mount_path="/calrissian"),
    persist=False,
)

## ConfigMaps

These configmaps are mounted as files on the pod.

### bash login

This file is used for the JupyterLab Terminal configuration

In [5]:
with open("config-maps/bash-login", "r") as f:
    content = f.read()

bash_login_cm = ConfigMap(
    name="bash-login",
    key="bash-login",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/workspace/.bash_login",
)

#### bash.rc


In [6]:
with open("config-maps/bash-rc", "r") as f:
    content = f.read()
bash_rc_cm = ConfigMap(
    name="bash-rc",
    key="bash-rc",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/workspace/.bashrc",
)

## bash login


In [7]:
with open("config-maps/bash-login", "r") as f:
    content = f.read()

bash_login_cm = ConfigMap(
    name="bash-login",
    key="bash-login",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/workspace/.bash_login",
)

## Profiles

In [8]:
profiles = []

### Coder

In [9]:
coders = {
    "coder1": {
        "display_name": "Code Server Small",
        "slug": "ellip_studio_coder_slug_s",
        "cpu_limit": 2,
        "mem_limit": "8G",
    },
    "coder2": {
        "display_name": "Code Server Medium",
        "slug": "ellip_studio_coder_slug_m",
        "cpu_limit": 4,
        "mem_limit": "12G",
    },
}

for key, value in coders.items():
    coder_definition = ProfileDefinition(
        display_name=value["display_name"],
        slug=value["slug"],
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_limit=value["cpu_limit"],
            mem_limit=value["mem_limit"],
            image="eoepca/pde-code-server:develop",
        ),
    )

    coder_profile = Profile(
        id=f"profile_studio_{key}",
        groups=["group-a", "group-b"],
        definition=coder_definition,
        node_selector={},
        volumes=[calrissian_volume, workspace_volume],
        config_maps=[
            bash_rc_cm,
        ],
        pod_env_vars={
            "HOME": "/workspace",
            "CONDA_ENVS_PATH": "/workspace/.envs",
        },
    )

    profiles.append(coder_profile)

## init.sh script

In [10]:
with open("./config-maps/init.sh", "r") as f:
    content = f.read()

init_cm = ConfigMap(
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
    default_mode="0660",
)


init_context_volume_mount = InitContainerVolumeMount(
    mount_path="/opt/init/.init.sh", name="init", sub_path="init"
)
init_container = InitContainer(
    name="init-file-on-volume",
    image="eoepca/pde-code-server:develop",
    command=["sh", "-c", "sh /opt/init/.init.sh"],
    volume_mounts=[
        VolumeMount(name="workspace-volume", mount_path="/workspace"),
        init_context_volume_mount,
    ],
)

eoepca_demo_init_script_profile = Profile(
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector={},
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)

In [11]:
profiles.append(eoepca_demo_init_script_profile)

eoepca_demo_init_script_profile = Profile(
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector={},
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)

## JupyterLab

In [12]:
image = "jupyter/scipy-notebook"


eoepca_jupyter_lab_profile = Profile(
    id="profile_jupyter_lab",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab",
        description="Jupyter Lab with Python 3.11",
        slug="eoepca_jupyter_lab",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
)

profiles.append(eoepca_jupyter_lab_profile)

## Image pull secret


In [13]:
image_pull_secret = ImagePullSecret(
    name="cr-config",
    persist=False,
    data="ewogICAgImF1dGhzIjogewogICAgICAgICJjci50ZXJyYWR1ZS5jb20iOiB7CiAgICAgICAgICAgICJ1c2VybmFtZSI6ICJyb2JvdCRlb2VwY2EtcGx1cy1ybyIsCiAgICAgICAgICAgICJwYXNzd29yZCI6ICJQMlE4TnkyZ0lHODhkZkxveXlLN05QVUZVbHJOekFZSiIsCiAgICAgICAgICAgICJlbWFpbCI6ICJlb2VwY2EtcGx1c0B0ZXJyYWR1ZS5jb20iLAogICAgICAgICAgICAiYXV0aCI6ICJjbTlpYjNRa1pXOWxjR05oTFhCc2RYTXRjbTg2VURKUk9FNTVNbWRKUnpnNFpHWk1iM2w1U3pkT1VGVkdWV3h5VG5wQldVbz0iCiAgICAgICAgfQogICAgfQp9",
)

In [14]:
image = "cr.terradue.com/eoepca-plus/scipy-notebook@sha256:f339a9fa98d3d0c1fa8d7cc850e7f5a46845781f49bee86aacba059669d02d54"
image = "eoepca/iat-jupyterlab:develop"

eoepca_jupyter_lab_profile_2 = Profile(
    id="profile_jupyter_lab_2",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab - profile 2",
        description="Jupyter Lab with Python 3.11 private image - demoes the use of an image pull secret",
        slug="eoepca_jupyter_lab_2",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
    image_pull_secrets=[image_pull_secret],
)

profiles.append(eoepca_jupyter_lab_profile_2)

## Stage-in/out

In [15]:
with open("./config-maps/init-stac.sh", "r") as f:
    content = f.read()

init_cm = ConfigMap(
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
)

In [16]:
with open("manifests/manifest.yaml", "r") as f:
    content = yaml.safe_load_all(f.read())


localstack_manifest = Manifest(
    name="manifests",
    key="manifests",
    readonly=True,
    persist=False,
    content=[e for e in content],
)

## e-learning

In [17]:
image = "docker.io/eoepca/pde-code-server@sha256:f57a3d5eabcae667e0db6e84a57b0c07c692c88f0fb5c8f6900ab8d5e38fcd40"

coder_profile_stac = Profile(
    id=f"profile_studio_coder_stac",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Understanding STAC for input/output data modelling",
        description="Understand the role of STAC in input/output data manifests in EO data processing workflows",
        slug="eoepca_coder_slug_stac",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[init_cm, bash_rc_cm, bash_login_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.local",
        "XDG_DATA_HOME": "/workspace/.local/share/",
        "CWLTOOL_OPTIONS": "--podman",
        "CODE_SERVER_WS": "/workspace/stac-eoap",
        "AWS_DEFAULT_REGION": "us-east-1",
        "AWS_ACCESS_KEY_ID": "test",
        "AWS_SECRET_ACCESS_KEY": "test",
    },
    role_bindings=[],
    init_containers=[init_container],
    image_pull_secrets=[image_pull_secret],
    manifests=[localstack_manifest],
)

profiles.append(coder_profile_stac)

## QGIS

In [18]:
with open("./config-maps/init-qgis.sh", "r") as f:
    content = f.read()

init_qgis_cm = ConfigMap(
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
)

image = "eoepca/iga-remote-desktop-qgis:1.1.3"

qgis_profile = Profile(
    id="profile_studio_desktop_qgis",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="QGIS on a Remote Desktop",
        description="Spatial visualization and decision-making tools for everyone",
        slug="eoepca_desktop_qgis",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_limit=2,
            mem_limit="2G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[bash_rc_cm, bash_login_cm],
    pod_env_vars={"HOME": "/workspace"},
    default_url="desktop",
    init_containers=[],
)

profiles.append(qgis_profile)

## Configuration

In [19]:
config = Config(profiles=profiles)

with open("../eoepca-demo/config.yml", "w") as file:
    yaml.dump(config.dict(), file)

In [20]:
# profiles